In [1]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'loan-approval-prediction:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F1226448%2F2047352%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240726%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240726T091029Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3Dc25b5c173efd320f43233026b5491abd21b5bc5fc1d25d29c61d96722be8ade924b7e03f63b14209376010594319a5f4c2dc5f7b76c93952bf646dbbf4bf30e5da6734d479a0f59790653de1b6caad23cb889628dca90184f7970162376b3dd6c27d29f9603c3152a4dd3c8cc7deac2a432d7cd9378aebad9e909e624e608ff428742484b17392517c0fee6f669b9bb2d2dd183a9d47eb2b7d66a92cd6aa35ed0cead269d50071b68fd6677a316a9587bf383a2eaaa6f0fd2d6c68d505853f49adb8bcec6250e4b4f8c5d3e9e3c8a9bd39396417c71c377264df2042d963ea86ea6a366346e1b47d69435c7c017a52eb95b2a0bdcdd7197a17446b464ebbcad0'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 13885 bytes downloaded
Downloaded and uncompressed: loan-approval-prediction
Data source import complete.


In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/loan-approval-prediction/Test Dataset.csv
/kaggle/input/loan-approval-prediction/Sample_Submission.csv
/kaggle/input/loan-approval-prediction/Training Dataset.csv


**Name :SIDDHARTH GARG**

## Loan Approval Pridiction Without EDA

## 1. Loading the dataset

In [3]:
df=pd.read_csv('/kaggle/input/loan-approval-prediction/Training Dataset.csv')
df

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...
609,LP002978,Female,No,0,Graduate,No,2900,0.0,71.0,360.0,1.0,Rural,Y
610,LP002979,Male,Yes,3+,Graduate,No,4106,0.0,40.0,180.0,1.0,Rural,Y
611,LP002983,Male,Yes,1,Graduate,No,8072,240.0,253.0,360.0,1.0,Urban,Y
612,LP002984,Male,Yes,2,Graduate,No,7583,0.0,187.0,360.0,1.0,Urban,Y


## 2. Analyze the dataset

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614 entries, 0 to 613
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Loan_ID            614 non-null    object 
 1   Gender             601 non-null    object 
 2   Married            611 non-null    object 
 3   Dependents         599 non-null    object 
 4   Education          614 non-null    object 
 5   Self_Employed      582 non-null    object 
 6   ApplicantIncome    614 non-null    int64  
 7   CoapplicantIncome  614 non-null    float64
 8   LoanAmount         592 non-null    float64
 9   Loan_Amount_Term   600 non-null    float64
 10  Credit_History     564 non-null    float64
 11  Property_Area      614 non-null    object 
 12  Loan_Status        614 non-null    object 
dtypes: float64(4), int64(1), object(8)
memory usage: 62.5+ KB


In [5]:
df.isnull().sum()

Loan_ID               0
Gender               13
Married               3
Dependents           15
Education             0
Self_Employed        32
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           22
Loan_Amount_Term     14
Credit_History       50
Property_Area         0
Loan_Status           0
dtype: int64

## 3. Handling Missing Values

In [6]:
print(df.columns)


Index(['Loan_ID', 'Gender', 'Married', 'Dependents', 'Education',
       'Self_Employed', 'ApplicantIncome', 'CoapplicantIncome', 'LoanAmount',
       'Loan_Amount_Term', 'Credit_History', 'Property_Area', 'Loan_Status'],
      dtype='object')


In [7]:
df.drop('Loan_ID', axis=1, inplace=True)

In [8]:
import numpy as np

from sklearn.impute import SimpleImputer

Imputer=SimpleImputer(missing_values=np.NaN,strategy="mean")
Imputer=Imputer.fit(df.iloc[:,7:10])
df.iloc[:,7:10]=Imputer.transform(df.iloc[:,7:10])

In [9]:
df.isnull().sum()

Gender               13
Married               3
Dependents           15
Education             0
Self_Employed        32
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount            0
Loan_Amount_Term      0
Credit_History        0
Property_Area         0
Loan_Status           0
dtype: int64

In [10]:
df=df.dropna()
df

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,Male,No,0,Graduate,No,5849,0.0,146.412162,360.0,1.0,Urban,Y
1,Male,Yes,1,Graduate,No,4583,1508.0,128.000000,360.0,1.0,Rural,N
2,Male,Yes,0,Graduate,Yes,3000,0.0,66.000000,360.0,1.0,Urban,Y
3,Male,Yes,0,Not Graduate,No,2583,2358.0,120.000000,360.0,1.0,Urban,Y
4,Male,No,0,Graduate,No,6000,0.0,141.000000,360.0,1.0,Urban,Y
...,...,...,...,...,...,...,...,...,...,...,...,...
609,Female,No,0,Graduate,No,2900,0.0,71.000000,360.0,1.0,Rural,Y
610,Male,Yes,3+,Graduate,No,4106,0.0,40.000000,180.0,1.0,Rural,Y
611,Male,Yes,1,Graduate,No,8072,240.0,253.000000,360.0,1.0,Urban,Y
612,Male,Yes,2,Graduate,No,7583,0.0,187.000000,360.0,1.0,Urban,Y


In [11]:
df.isnull().sum()

Gender               0
Married              0
Dependents           0
Education            0
Self_Employed        0
ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           0
Loan_Amount_Term     0
Credit_History       0
Property_Area        0
Loan_Status          0
dtype: int64

In [12]:
# ['Gender'],['Married'],['Education'],['Self_Employed'],['Property_Area'],['Loan_Status']

Categorical_Columns=['Property_Area']


## 4. Encode Categorical Variables

In [13]:
df.describe

<bound method NDFrame.describe of      Gender Married Dependents     Education Self_Employed  ApplicantIncome  \
0      Male      No          0      Graduate            No             5849   
1      Male     Yes          1      Graduate            No             4583   
2      Male     Yes          0      Graduate           Yes             3000   
3      Male     Yes          0  Not Graduate            No             2583   
4      Male      No          0      Graduate            No             6000   
..      ...     ...        ...           ...           ...              ...   
609  Female      No          0      Graduate            No             2900   
610    Male     Yes         3+      Graduate            No             4106   
611    Male     Yes          1      Graduate            No             8072   
612    Male     Yes          2      Graduate            No             7583   
613  Female      No          0      Graduate           Yes             4583   

     CoapplicantIncome  LoanAmount  Loan_Amount_Term  Credit_History  \
0                  0.0  146.412162             360.0             1.0   
1               1508.0  128.000000             360.0             1.0   
2                  0.0   66.000000             360.0             1.0   
3               2358.0  120.000000             360.0             1.0   
4                  0.0  141.000000             360.0             1.0   
..                 ...         ...               ...             ...   
609                0.0   71.000000             360.0             1.0   
610                0.0   40.000000             180.0             1.0   
611              240.0  253.000000             360.0             1.0   
612                0.0  187.000000             360.0             1.0   
613                0.0  133.000000             360.0             0.0   

    Property_Area Loan_Status  
0           Urban           Y  
1           Rural           N  
2           Urban           Y  
3           Urban           Y  
4           Urban           Y  
..            ...         ...  
609         Rural           Y  
610         Rural           Y  
611         Urban           Y  
612         Urban           Y  
613     Semiurban           N  

[554 rows x 12 columns]>

In [14]:
from sklearn.preprocessing import LabelEncoder
LE = LabelEncoder()
df['Gender'] = LE.fit_transform(df['Gender'])
df['Married'] = LE.fit_transform(df['Married'])
df['Education'] = LE.fit_transform(df['Education'])
df['Self_Employed'] = LE.fit_transform(df['Self_Employed'])
df['Loan_Status'] = LE.fit_transform(df['Loan_Status'])

<ipython-input-14-3924e7aba36c>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Gender'] = LE.fit_transform(df['Gender'])
<ipython-input-14-3924e7aba36c>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Married'] = LE.fit_transform(df['Married'])
<ipython-input-14-3924e7aba36c>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-d

In [15]:
df

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,1,0,0,0,0,5849,0.0,146.412162,360.0,1.0,Urban,1
1,1,1,1,0,0,4583,1508.0,128.000000,360.0,1.0,Rural,0
2,1,1,0,0,1,3000,0.0,66.000000,360.0,1.0,Urban,1
3,1,1,0,1,0,2583,2358.0,120.000000,360.0,1.0,Urban,1
4,1,0,0,0,0,6000,0.0,141.000000,360.0,1.0,Urban,1
...,...,...,...,...,...,...,...,...,...,...,...,...
609,0,0,0,0,0,2900,0.0,71.000000,360.0,1.0,Rural,1
610,1,1,3+,0,0,4106,0.0,40.000000,180.0,1.0,Rural,1
611,1,1,1,0,0,8072,240.0,253.000000,360.0,1.0,Urban,1
612,1,1,2,0,0,7583,0.0,187.000000,360.0,1.0,Urban,1


In [16]:
# Convert to numeric and replace errors with 4
df['Dependents'] = pd.to_numeric(df['Dependents'], errors='coerce').fillna(4).astype(int)

<ipython-input-16-a681f2b8f919>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Dependents'] = pd.to_numeric(df['Dependents'], errors='coerce').fillna(4).astype(int)


In [17]:
df=pd.get_dummies(data=df,columns=['Property_Area'],drop_first=True,dtype=int)
df

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Loan_Status,Property_Area_Semiurban,Property_Area_Urban
0,1,0,0,0,0,5849,0.0,146.412162,360.0,1.0,1,0,1
1,1,1,1,0,0,4583,1508.0,128.000000,360.0,1.0,0,0,0
2,1,1,0,0,1,3000,0.0,66.000000,360.0,1.0,1,0,1
3,1,1,0,1,0,2583,2358.0,120.000000,360.0,1.0,1,0,1
4,1,0,0,0,0,6000,0.0,141.000000,360.0,1.0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
609,0,0,0,0,0,2900,0.0,71.000000,360.0,1.0,1,0,0
610,1,1,4,0,0,4106,0.0,40.000000,180.0,1.0,1,0,0
611,1,1,1,0,0,8072,240.0,253.000000,360.0,1.0,1,0,1
612,1,1,2,0,0,7583,0.0,187.000000,360.0,1.0,1,0,1


In [18]:
df.head()

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Loan_Status,Property_Area_Semiurban,Property_Area_Urban
0,1,0,0,0,0,5849,0.0,146.412162,360.0,1.0,1,0,1
1,1,1,1,0,0,4583,1508.0,128.000000,360.0,1.0,0,0,0
2,1,1,0,0,1,3000,0.0,66.000000,360.0,1.0,1,0,1
3,1,1,0,1,0,2583,2358.0,120.000000,360.0,1.0,1,0,1
4,1,0,0,0,0,6000,0.0,141.000000,360.0,1.0,1,0,1


## 5. Handling outliers for 'ApplicantIncome' Column

In [19]:
def handling_outliers(dataset):
    quantile_25 = dataset['ApplicantIncome'].astype(float).quantile(0.25)
    quantile_75 = dataset['ApplicantIncome'].astype(float).quantile(0.75)
    iqr = quantile_75 - quantile_25

    lower_percentile = quantile_25 - 1.5 * iqr
    upper_percentile = quantile_75 + 1.5 * iqr

    dataset = dataset[dataset['ApplicantIncome'].astype(float) >= lower_percentile]
    dataset = dataset[dataset['ApplicantIncome'].astype(float) <= upper_percentile]

    return dataset

In [20]:
df=handling_outliers(df)

In [21]:
df

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Loan_Status,Property_Area_Semiurban,Property_Area_Urban
0,1,0,0,0,0,5849,0.0,146.412162,360.0,1.0,1,0,1
1,1,1,1,0,0,4583,1508.0,128.000000,360.0,1.0,0,0,0
2,1,1,0,0,1,3000,0.0,66.000000,360.0,1.0,1,0,1
3,1,1,0,1,0,2583,2358.0,120.000000,360.0,1.0,1,0,1
4,1,0,0,0,0,6000,0.0,141.000000,360.0,1.0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
609,0,0,0,0,0,2900,0.0,71.000000,360.0,1.0,1,0,0
610,1,1,4,0,0,4106,0.0,40.000000,180.0,1.0,1,0,0
611,1,1,1,0,0,8072,240.0,253.000000,360.0,1.0,1,0,1
612,1,1,2,0,0,7583,0.0,187.000000,360.0,1.0,1,0,1


In [22]:
# separate dependent variable

y=df.pop('Loan_Status')

## 6. Transform into same scale

In [23]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
X=sc.fit_transform(df)


In [24]:
X

array([[ 0.46914357, -1.37162529, -0.68872555, ...,  0.44724508,
        -0.78679579,  1.46565622],
       [ 0.46914357,  0.72906209,  0.13580504, ...,  0.44724508,
        -0.78679579, -0.68228824],
       [ 0.46914357,  0.72906209, -0.68872555, ...,  0.44724508,
        -0.78679579,  1.46565622],
       ...,
       [ 0.46914357,  0.72906209,  0.13580504, ...,  0.44724508,
        -0.78679579,  1.46565622],
       [ 0.46914357,  0.72906209,  0.96033563, ...,  0.44724508,
        -0.78679579,  1.46565622],
       [-2.13154363, -1.37162529, -0.68872555, ..., -2.42867315,
         1.27097782, -0.68228824]])

In [25]:
y

0      1
1      0
2      1
3      1
4      1
      ..
609    1
610    1
611    1
612    1
613    0
Name: Loan_Status, Length: 510, dtype: int64

## 7. Train-Test Split

In [26]:
df

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area_Semiurban,Property_Area_Urban
0,1,0,0,0,0,5849,0.0,146.412162,360.0,1.0,0,1
1,1,1,1,0,0,4583,1508.0,128.000000,360.0,1.0,0,0
2,1,1,0,0,1,3000,0.0,66.000000,360.0,1.0,0,1
3,1,1,0,1,0,2583,2358.0,120.000000,360.0,1.0,0,1
4,1,0,0,0,0,6000,0.0,141.000000,360.0,1.0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
609,0,0,0,0,0,2900,0.0,71.000000,360.0,1.0,0,0
610,1,1,4,0,0,4106,0.0,40.000000,180.0,1.0,0,0
611,1,1,1,0,0,8072,240.0,253.000000,360.0,1.0,0,1
612,1,1,2,0,0,7583,0.0,187.000000,360.0,1.0,0,1


In [27]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=0)

In [28]:
X_train

array([[ 0.46914357,  0.72906209, -0.68872555, ..., -2.42867315,
        -0.78679579,  1.46565622],
       [ 0.46914357,  0.72906209, -0.68872555, ..., -0.0065789 ,
        -0.78679579, -0.68228824],
       [ 0.46914357,  0.72906209,  0.13580504, ...,  0.44724508,
         1.27097782, -0.68228824],
       ...,
       [ 0.46914357, -1.37162529, -0.68872555, ...,  0.44724508,
        -0.78679579,  1.46565622],
       [ 0.46914357,  0.72906209, -0.68872555, ...,  0.44724508,
         1.27097782, -0.68228824],
       [ 0.46914357, -1.37162529, -0.68872555, ...,  0.44724508,
        -0.78679579, -0.68228824]])

In [29]:
X_test

array([[ 0.46914357, -1.37162529, -0.68872555, ...,  0.44724508,
         1.27097782, -0.68228824],
       [ 0.46914357,  0.72906209, -0.68872555, ..., -0.0065789 ,
        -0.78679579, -0.68228824],
       [ 0.46914357, -1.37162529, -0.68872555, ...,  0.44724508,
        -0.78679579, -0.68228824],
       ...,
       [ 0.46914357,  0.72906209,  0.96033563, ...,  0.44724508,
         1.27097782, -0.68228824],
       [ 0.46914357,  0.72906209, -0.68872555, ...,  0.44724508,
        -0.78679579,  1.46565622],
       [-2.13154363, -1.37162529,  0.13580504, ...,  0.44724508,
         1.27097782, -0.68228824]])

In [30]:
y_train

387    0
444    1
81     1
472    1
6      1
      ..
388    1
230    1
141    1
56     1
207    1
Name: Loan_Status, Length: 408, dtype: int64

In [31]:
y_test

101    1
129    0
401    0
602    1
248    1
      ..
450    0
65     0
520    1
70     1
410    0
Name: Loan_Status, Length: 102, dtype: int64

## 8. Creating Model

In [32]:
from sklearn.linear_model import LogisticRegression

# Train the Logistic Regression model
model = LogisticRegression()
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)



In [33]:
y_pred

array([1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1])

In [34]:
y_test

101    1
129    0
401    0
602    1
248    1
      ..
450    0
65     0
520    1
70     1
410    0
Name: Loan_Status, Length: 102, dtype: int64

## Evaluate the Model

In [35]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report


# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

print(f'Accuracy: {accuracy}')
print(f'Confusion Matrix:\n{conf_matrix}')
print(f'Classification Report:\n{class_report}')

Accuracy: 0.7941176470588235
Confusion Matrix:
[[15 20]
 [ 1 66]]
Classification Report:
              precision    recall  f1-score   support

           0       0.94      0.43      0.59        35
           1       0.77      0.99      0.86        67

    accuracy                           0.79       102
   macro avg       0.85      0.71      0.73       102
weighted avg       0.83      0.79      0.77       102



**See Both results**

In [36]:
# Set display options to show max rows
pd.set_option('display.max_rows', None)

# Create the DataFrame with true and predicted labels
result_df = pd.DataFrame({'True Label': y_test, 'Predicted Label': y_pred})

# Display the DataFrame
print(result_df)


     True Label  Predicted Label
101           1                1
129           0                1
401           0                1
602           1                1
248           1                1
17            0                0
524           0                1
340           0                1
196           1                1
235           1                1
245           0                1
246           1                1
53            0                1
266           1                1
186           0                0
45            1                1
390           1                1
338           0                0
492           1                1
551           1                1
372           1                1
359           1                1
348           1                1
131           1                1
172           0                1
55            1                1
362           1                1
26            1                1
480           1                1
315       